In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import joblib

In [2]:
queries_df = pd.read_csv('dataset_users_query.csv')
answers_df = pd.read_csv('answers.csv')

queries_df['query'] = queries_df['query'].str.lower().str.strip()
answers_df['query'] = answers_df['query'].str.lower().str.strip()

employment_df = answers_df.drop(['Unnamed: 0', 'по должности-лемме', 'по дополнительному признаку', 'по условиям', 'общие фразы'], axis = 1)
position_df = answers_df.drop(['Unnamed: 0', 'занятость', 'по дополнительному признаку', 'по условиям', 'общие фразы'], axis = 1)
additance_df = answers_df.drop(['Unnamed: 0', 'занятость', 'по должности-лемме', 'по условиям', 'общие фразы'], axis = 1)
conditions_df = answers_df.drop(['Unnamed: 0', 'занятость', 'по должности-лемме', 'по дополнительному признаку', 'общие фразы'], axis = 1)
phrase_df = answers_df.drop(['Unnamed: 0', 'занятость', 'по должности-лемме', 'по дополнительному признаку', 'по условиям'], axis = 1)

In [3]:
employment_keywords = list(answers_df['занятость'].unique()[1:])
position_keywords = list(answers_df['по должности-лемме'].unique()[1:])
additance_keywords = list(answers_df['по дополнительному признаку'].unique()[1:])
conditions_keywords = list(answers_df['по условиям'].unique()[1:])
common_phrases = ['работа', 'вакансии']

In [13]:
answers_df

,Unnamed: 0,query,занятость,по должности-лемме,по дополнительному признаку,по условиям,общие фразы
0,0,фарпост работа владивосток,NaN,NaN,NaN,NaN,общая фраза
1,1,кофейни вакансии,NaN,NaN,NaN,NaN,общая фраза
2,2,работа разнорабочие часовой,NaN,Рабочий,NaN,NaN,NaN
3,3,личный водитель на день,на неполный день,Водитель,NaN,NaN,NaN
4,4,работа от работодателя персональный водитель,NaN,Водитель,NaN,NaN,NaN
...,...,...,...,...,...,...,...
14240,14240,япония вакансии,NaN,NaN,NaN,NaN,общая фраза
14241,14241,япония работа,NaN,NaN,NaN,NaN,общая фраза
14242,14242,японский язык работа,NaN,NaN,NaN,NaN,общая фраза
14243,14243,яппи вакансии,NaN,NaN,NaN,NaN,общая фраза


In [4]:
encoder = LabelEncoder()

additance_df['category_encoded'] = encoder.fit_transform(additance_df['по дополнительному признаку'])

In [5]:
label_dict = {i: class_name for i, class_name in enumerate(encoder.classes_)}

In [6]:
vectorizer = TfidfVectorizer(max_features=5000)

X = vectorizer.fit_transform(additance_df['query'])
y = additance_df['category_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
model_query = LogisticRegression(max_iter=1000)
model_query.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [8]:
y_pred = model_query.predict(X_test)

In [9]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99        78
           3       0.96      0.85      0.90        26
           4       0.00      0.00      0.00         1
           6       1.00      0.50      0.67         2
           7       1.00      0.55      0.71        11
           9       1.00      0.50      0.67        12
          10       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          13       0.94      0.81      0.87        21
          15       0.00      0.00      0.00         1
          16       1.00      0.86      0.92         7
          18       0.99      1.00      1.00      2688

    accuracy                           0.99      2849
   macro avg       0.66      0.50      0.56      2849
weighted avg       0.99      0.99      0.99      2849



C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
joblib.dump(model_query, 'additance_model.pkl')

['additance_model.pkl']

In [11]:
joblib.dump(vectorizer, 'additance_vectorizer.pkl')

['additance_vectorizer.pkl']